In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

# Example: Replace these with actual paths to your datasets
csic_2010_path = 'C:\prateek\projects\waf\csic_final.csv'
pkdd_2007_path = ''

# Load datasets
csic_2010_df = pd.read_csv(csic_2010_path)
pkdd_2007_df = pd.read_csv(pkdd_2007_path)
